In [1]:
'''
This runs a forecast pipeline using both python and R

It uses a hypbrid approach with two R forecasters that worked well in the past as described in
https://robjhyndman.com/hyndsight/show-me-the-evidence/. It combines ETS and auto.ARIMA outputs equally weighted.


1. Important R packages required in conda include: r-essentials, r-forecast, r-tseries, rpy2
   I had to install the R package Mcomp in R using the command: install.packages("Mcomp") which interacts with a user
   to select a mirror site.
   
2. This works in a conda environment with python 3.6.3, numpy, pandas, and other normal packages on windows 10 64 bits
   with Anaconda 3 5.01 64 bit 
   
3. It needs no special arguments but will find the data and run a prediction based on the dataset file structure assuming 
   that the data, problem, and solution directories are at the same level below the dataset root directory.
   
4. This pipeline also assumes a forecasting problem with the data to forecast at the end of the provided data as
   specified in the splits file.

'''

# need this to run both python and R
%load_ext rpy2.ipython

In [2]:
# these R libraries are necessary for advanced time series forecasting
%R library(forecast)
%R library(tseries)

array(['tseries', 'forecast', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

In [3]:
# find paths to data schema, problem schema, data, and splits file
import os, ntpath
from pathlib import Path
import sys, json
import pandas as pd
import numpy as np

# get names of the three main dataset directories
path = os.getcwd()
solution_dir_name = ntpath.basename(path)
data_dir_name = solution_dir_name.replace("solution", "dataset")
problem_dir_name = solution_dir_name.replace("solution", "problem")

solution_dir_path = os.getcwd()
ds = "../" + data_dir_name
data_dir_path = Path(ds).resolve()
ps = "../" + problem_dir_name
problem_dir_path = Path(ps).resolve()


print("data directory: %s\nproblem directory %s\nsolution directory %s" % (data_dir_path, problem_dir_path, solution_dir_path))

data directory: C:\Users\cllippmann\d3m\datasets\LL1_fluid_milk_wholesale_price_all_industry\LL1_fluid_milk_wholesale_price_all_industry_dataset
problem directory C:\Users\cllippmann\d3m\datasets\LL1_fluid_milk_wholesale_price_all_industry\LL1_fluid_milk_wholesale_price_all_industry_problem
solution directory C:\Users\cllippmann\d3m\datasets\LL1_fluid_milk_wholesale_price_all_industry\LL1_fluid_milk_wholesale_price_all_industry_solution


In [4]:
#%pylab
# read in all data assuming this is time series data where the training data comes first and the test data follows
learning_data_path = Path(str(data_dir_path) + "/tables/learningData.csv").resolve()
print(learning_data_path)
df_data = pd.read_csv(learning_data_path, index_col='d3mIndex')
all_data = np.asarray(df_data["value"])

# get train/test splits
splits_data_path = Path(str(problem_dir_path) + "/dataSplits.csv").resolve()
print(splits_data_path)

df_splits = pd.read_csv(splits_data_path, index_col='d3mIndex')
train_flags = df_splits[df_splits['type']=='TRAIN']
test_flags = df_splits[df_splits['type']=='TEST']
xlen = len(train_flags)
xxlen = len(test_flags)

x = np.zeros(xlen)
xx = np.zeros(xxlen)
x=all_data[0:xlen]
xx = all_data[xlen:]
hh = len(xx)
#plot(x)

C:\Users\cllippmann\d3m\datasets\LL1_fluid_milk_wholesale_price_all_industry\LL1_fluid_milk_wholesale_price_all_industry_dataset\tables\learningData.csv
C:\Users\cllippmann\d3m\datasets\LL1_fluid_milk_wholesale_price_all_industry\LL1_fluid_milk_wholesale_price_all_industry_problem\dataSplits.csv


In [5]:
%%R -i x,hh -o p1,p2 -o y
p1 = forecast(auto.arima(x),h=hh)$mean
p2 = forecast(ets(x), h=hh)$mean
y = 0.5*(p1+p2)

In [6]:
print(p1, p2)

Time Series:

Start = 116 

End = 133 

Frequency = 1 

 [1] 2678.470 2657.861 2637.896 2618.626 2603.679 2594.028 2589.087 2587.582

 [9] 2588.154 2589.677 2591.374 2592.812 2593.821 2594.402 2594.643 2594.660

[17] 2594.559 2594.417
 Time Series:

Start = 116 

End = 133 

Frequency = 1 

 [1] 2712.064 2737.681 2758.174 2774.569 2787.684 2798.177 2806.571 2813.286

 [9] 2818.658 2822.956 2826.394 2829.145 2831.345 2833.106 2834.514 2835.641

[17] 2836.542 2837.263



In [7]:
%%R -i x,p1,p2,xlen,xxlen
jpeg('forecast-plot.png')
plot(x,xlim=c(0,xlen+xxlen))
lines(x)
lines(p1,col='red')
lines(p2,col='blue')
lines(y,col='green')
dev.off()

In [8]:
# compute error on test data
from collections import OrderedDict
from sklearn.metrics import mean_squared_error

y_truth = xx
y_predicted = np.array(y)
rmse = np.sqrt(mean_squared_error(y_truth, y_predicted))
print('performance on test data (rmse):',rmse)
print('saving the performance score ...')
test_performance = OrderedDict([
    ('test', OrderedDict([
        ('score', OrderedDict([
                ('metric', 'rootMeanSquaredError'),
                ('value', rmse)])
        )
    ]))
])

performance on test data (rmse): 138.943969746
saving the performance score ...


In [9]:
# create scores file
overall_performance = OrderedDict()
overall_performance.update(test_performance)

with open('scores.json', 'w', encoding='utf-8') as f:
    json.dump(overall_performance, f, indent=2)
print(json.dumps(overall_performance, indent=2))

{
  "test": {
    "score": {
      "metric": "rootMeanSquaredError",
      "value": 138.943969746418
    }
  }
}


In [10]:
# create predictions file
test_flags
preds= pd.Series(xx)
test_flags = test_flags.assign(a=preds.values)
test_flags.rename(columns={'a': '0'}, inplace=True)
test_flags.to_csv("predictions.csv")